In [1]:
import mysql.connector as ms
import numpy as np
import pandas as pd
import requests
import json as oj
import os
import traceback
import warnings
import logging
from datetime import datetime, timedelta, timezone

warnings.filterwarnings("ignore")
logger = logging.getLogger(__name__)

/Users/anahita/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
DB_MYSQL_HOST = 'data-analytics-dev.test.skeps.dev'
DB_MYSQL_PORT = int(3376)
DB_MYSQL_USER = 'chainwolf'
DB_MYSQL_PASS = 'mysqlchainwolf.911'

In [3]:
url = "https://ucl-fnbo.test.skeps.dev"

In [4]:
def error_case(generic_error,error_service,order_id = None):
    current_date= datetime.now().date().strftime('%Y-%m-%d')
    log_file_path = "Error_"+current_date+".txt"

    with open(log_file_path, 'a') as log_file:
        log_file.write(f"Exception: {error_service} File : {str(generic_error)} : order_id = {order_id} \n")

def none_to_nan(df):
    return df.replace([None], np.nan, inplace = True)

def write_files(data_dfs, filenames, abc_filenames, path):

    for df, filename,abc_filename in zip(data_dfs, filenames, abc_filenames):

        df.to_csv(path+'/'+filename,index=False,sep='|')

        if filename[-4:]=='.txt':
            with open(path+'/'+filename) as f:
                lines = f.readlines()
                # print(len(lines))
                if len(lines)>0:
                    last = len(lines) - 1
                    lines[last] = lines[last].replace('\r','').replace('\n','')
            with open(path+'/'+filename, 'w') as wr:
                wr.writelines(lines)

        if abc_filename!="":
            var1 = 'VERSION 1.0'
            var2 = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")
            var3 = 'COUNT'
            var4 = df.shape[0]

            data = [var1, var2, var3, var4]
            # Specify the file path with a .abc extension
            file_path = path+'/'+abc_filename

            # Write the string to the file
            with open(file_path, 'w') as file:
                file.write(str(var1))
                file.write('\n')  # Add a newline for separation
                file.write(str(var2))
                file.write('\n')  # Add a newline for separation
                file.write(str(var3))
                file.write('\n')  # Add a newline for separation
                file.write(str(var4))

def create_folder_if_not_exists(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created.")
    else:
        print(f"Folder '{folder_name}' already exists.")

def get_api_data(url, params=None, headers=None):
    """
    Fetch data from API using requests and parse with orjson
    """
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()  # Raise exception for bad status codes
        return oj.loads(response.text)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
def transform_citizenship_status(raw_status):
    if raw_status == "PERMANENT":
        return "Yes"
    elif raw_status == "TEMPORARY":
        return "No"
    else:
        return ""
    
def get_pii_data(url, app_id):
    try:
        api_url = url+"/application/internal/reader/customer/business/"+app_id
        params={"type": "APPLICANT_INFO"}
        resp = get_api_data(api_url, params=params)
        resp_json = resp.get('response',{}) if pd.notnull(resp) else dict()
        bor_phone = resp_json.get("cell_phone","")
        bor_ssn = resp_json.get("ssn","")
        bor_first_name = resp_json.get("first_name","")
        bor_middle_name = resp_json.get("middle_name","")
        bor_last_name = resp_json.get("last_name","")
        bor_email = resp_json.get("email","")
        bor_address = resp_json.get("address",{}).get("street_address_1","")
        bor_address_line2 = resp_json.get("address",{}).get("street_address_2","")
        bor_city = resp_json.get("address",{}).get("city","")
        bor_state = resp_json.get("address",{}).get("state","")
        bor_zip = resp_json.get("address",{}).get("zip_code","")
        bor_dob = resp_json.get("date_of_birth","")
        bor_income = resp_json.get("financial_details",{}).get("annual_income","")
        bor_us_citznshp_status = transform_citizenship_status(resp_json.get("citizenship","")[0].get("status","") if pd.notna(resp_json.get("citizenship",None)) else "")
        bor_residence_type = resp_json.get("address",{}).get("ownership_type","")
        bor_mother_maiden_name = resp_json.get("family_details","")[0].get("maiden_name","") if pd.notna(resp_json.get("family_details",None)) else ""
        return [bor_phone,bor_ssn,bor_first_name,bor_middle_name,bor_last_name,bor_email,bor_address,bor_address_line2,bor_city,bor_state,bor_zip,bor_dob,bor_income,bor_us_citznshp_status,bor_residence_type,bor_mother_maiden_name]
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return ["","","","","","","","","","","","","","","",""]
    


In [35]:

def error_case(generic_error,error_service,order_id = None):
    current_date= datetime.now().date().strftime('%Y-%m-%d')
    log_file_path = "Error_"+current_date+".txt"

    with open(log_file_path, 'a') as log_file:
        log_file.write(f"Exception: {error_service} File : {str(generic_error)} : order_id = {order_id} \n")


def date_to_str(in_dt):
    if isinstance(in_dt, (date)):
        return in_dt.strftime("%Y-%m-%d")
    else:
        return "Not date"

def align_date(value):
    if value is not None:
        if "/" in value:
            return pd.to_datetime(value[:10],format='mixed').strftime("%m/%d/%Y")
        elif "-" in value:
            return pd.to_datetime(value[:10],format='mixed').strftime("%m/%d/%Y")
    else:
        return None

def format_start_mmyyyy(in_start_mmyyyy):
    if "/" in in_start_mmyyyy:
        return in_start_mmyyyy
    elif in_start_mmyyyy=='None':
        return ""
    else:
        return in_start_mmyyyy[:2]+"/"+in_start_mmyyyy[2:]


def write_log_count_file_ucl(start_date, end_date, Number_Of_Records, flag, filetype):

    log_file = pd.DataFrame(columns=['RowNumber','start_date','end_date','creation_date','Number_Of_Records','Success_Flag'])
    log_file.loc[0,'start_date']= start_date
    log_file.loc[0,'end_date'] = end_date
    log_file.loc[0,'creation_date'] =  datetime.now().date()
    log_file.loc[0,'Number_Of_Records'] =  Number_Of_Records
    log_file.loc[0,'Success_Flag'] =  flag

    if filetype.upper() == 'MERCHANT':
        file_path = "Logs_UCL.csv"
    else:
        return

    # Check if the file exists
    if os.path.exists(file_path):
        # Read the Excel file
        df = pd.read_csv(file_path)
        last_RowNumber = df['RowNumber'].dropna().iloc[-1]
        log_file.loc[0,'RowNumber'] = last_RowNumber+1
        log_file.to_csv(file_path, mode='a', header=False, index=False)
    else:
        print("File does not exist.")
        log_file.loc[0,'RowNumber'] = 1
        log_file.to_csv(file_path,index=False)



def get_start_date_ucl():

    file_path = "Logs_UCL.csv"

    if os.path.exists(file_path):
        # Read the Excel file
        df = pd.read_csv(file_path)

        filtered_df = df[df['Success_Flag'] == 1]

        # Check if there are any matching rows
        if not filtered_df.empty:
            # Get the value of column1 from the last matching row
            end_date_of_last_success = filtered_df.iloc[-1]['end_date']
            if end_date_of_last_success == (datetime.now() - timedelta(days=1)).date().strftime('%Y-%m-%d') :
                return  (datetime.now()- timedelta(days=1)).date().strftime('%Y-%m-%d')
            else:
                new_start_date = (datetime.strptime(end_date_of_last_success, '%Y-%m-%d') + timedelta(days=1)).date()
                new_start_date = new_start_date.strftime('%Y-%m-%d')
                return new_start_date

        else:
            # print("No records found where column2 equals 1.")
            return (datetime.now()- timedelta(days=1)).date().strftime('%Y-%m-%d')

    else:
        return (datetime.now()- timedelta(days=1)).date().strftime('%Y-%m-%d')

def get_prove_data(data):
    """
    Fetch Prove data from API response
    """

    prove_request, prove_response = None, None

    try:

        main_prove = data.get("response", {}).get("PAYFONE", {})
        # print("Main Prove Is", main_prove)

        key = list(main_prove.keys())[0]
        # print(key)

        # prove_request
        prove_request = oj.dumps(main_prove.get(key, {})[0].get("request", {}), ensure_ascii=True)

        # prove_response
        prove_response = oj.dumps(main_prove.get(key, {})[0].get("response", {}), ensure_ascii=True)

    
    except Exception as e:
        pass
    
    return prove_request, prove_response

def get_plaid_data(data):
    """
    Fetch Plaid data from API response
    """

    plaid_identity_request, plaid_identity_response = None, None

    # try:

    main_plaid = data.get("response", {}).get("PLAID", {})
    # print("Main Plaid Is", main_plaid)

    for val in list(main_plaid.values()):
        for data in val:
            # plaid_identity_request
            # plaid_identity_response
            if data['request_identifier'] == 'GET_IDENTITY':
                plaid_identity_request = oj.dumps(data['request'], ensure_ascii=True)
                plaid_identity_response = oj.dumps(data['response'], ensure_ascii=True)

    
    # except Exception as e:
    #     print(f"Encountered Exception: {e} for Plaid")
    
    return plaid_identity_request, plaid_identity_response

def get_provenir_data(data):
    """
    Fetch Provenir data from API response
    """

    provenir_verification_request, provenir_verification_response, \
        provenir_credit_check_request, provenir_credit_check_response, \
        provenir_bank_account_check_request, provenir_bank_account_check_response, \
        provenir_final_terms_check_request, provenir_final_terms_check_response, \
        provenir_applicaton_update_check_request, provenir_applicaton_update_check_response = (None,) * 10

    # try:

    main_provenir = data.get("response", {}).get("PROVENIR", {})
    
    for val in list(main_provenir.values()):
        for data in val:
            # provenir_verification_request
            # provenir_verification_response
            if data['request_identifier'] == 'PROVENIR_VERIFICATION':
                provenir_verification_request = oj.dumps(data['request'], ensure_ascii=True)
                provenir_verification_response = oj.dumps(data['response'], ensure_ascii=True)

            # provenir_credit_check_request
            # provenir_credit_check_response
            elif data['request_identifier'] == 'PROVENIR_CREDIT_CHECK':
                provenir_credit_check_request = oj.dumps(data['request'], ensure_ascii=True)
                provenir_credit_check_response = oj.dumps(data['response'], ensure_ascii=True)

            # provenir_bank_account_check_request
            # provenir_bank_account_check_response
            elif data['request_identifier'] == 'PROVENIR_BANK_ACCOUNT_CHECK':
                provenir_bank_account_check_request = oj.dumps(data['request'], ensure_ascii=True)
                provenir_bank_account_check_response = oj.dumps(data['response'], ensure_ascii=True)

            # provenir_final_terms_check_request
            # provenir_final_terms_check_response
            elif data['request_identifier'] == 'PROVENIR_FINAL_TERMS':
                provenir_final_terms_check_request = oj.dumps(data['request'], ensure_ascii=True)
                provenir_final_terms_check_response = oj.dumps(data['response'], ensure_ascii=True)

            # provenir_applicaton_update_check_request
            # provenir_applicaton_update_check_response
            elif data['request_identifier'] == 'PROVENIR_APPLICATION_UPDATE':
                provenir_applicaton_update_check_request = oj.dumps(data['request'], ensure_ascii=True)
                provenir_applicaton_update_check_response = oj.dumps(data['response'], ensure_ascii=True)
        
        
    # except Exception as e:
    #     print(f"Encountered Exception: {e} for Provenir")


    return provenir_verification_request, provenir_verification_response, \
        provenir_credit_check_request, provenir_credit_check_response, \
        provenir_bank_account_check_request, provenir_bank_account_check_response, \
        provenir_final_terms_check_request, provenir_final_terms_check_response, \
        provenir_applicaton_update_check_request, provenir_applicaton_update_check_response

def get_prove_ssn_dob(id,id_type):
    api_url = url+"/application/logs/vendor/"
    prove_request_2, prove_response_2 = get_prove_data(get_api_data(api_url, params={"id": id, "id_type": id_type}))
    if pd.notna(prove_response_2):
        prove_response_js = oj.loads(prove_response_2)
        prove_ssn = prove_response_js.get("body",{}).get("response",{}).get("individual",{}).get("ssn",{})
        prove_dob = prove_response_js.get("body",{}).get("response",{}).get("individual",{}).get("dob",{})
    else:
        prove_ssn = ""
        prove_dob = ""
    return prove_ssn, prove_dob

def get_match_ind (var1,var2):
    if (pd.isna(var1)) | (pd.isna(var2)) | (var2=="") | (var1==""):
        return ""
    else:
        return int(var1==var2)

In [47]:
def main_func(run_type, env, start_date = None, end_date = None):
    try:
        # current_directory = Config.BASE_DIR
        current_directory = os.curdir
        print("📂 current_directory ➟ ", current_directory)
        logger.info("📂 current_directory ➟ %s", current_directory)

        host = DB_MYSQL_HOST
        port = int(DB_MYSQL_PORT)
        user = DB_MYSQL_USER
        password = DB_MYSQL_PASS

        create_folder_if_not_exists('ucl_data_files')
        new_folder_path = os.path.join(current_directory, 'ucl_data_files')
        print("📂 new_folder_path ➟ ", new_folder_path)
        logger.info("📂 new_folder_path ➟ %s", new_folder_path)
        os.chdir(new_folder_path)

        if run_type.upper() == "AUTO" or  (start_date == None and end_date == None):
            start_date = get_start_date_ucl()
            end_date = datetime.now().date().strftime('%Y-%m-%d')

        formatted_start_date = datetime.strptime(str(start_date), "%Y-%m-%d").strftime("%Y_%m_%d")
        formatted_end_date = datetime.strptime(str(end_date), "%Y-%m-%d").strftime("%Y_%m_%d")
        folder_name = formatted_start_date + "_" + formatted_end_date
        print("📂 folder_name ➟ ", folder_name)
        logger.info("📂 folder_name ➟ %s", folder_name)
        create_folder_if_not_exists(folder_name)

        ## SQL Queries
        txn_vars = ["transaction_id","promo_segment_id","transaction_datetime","merchant_id","application_id","loan_purpose","promo_code","solicitation_id","promo_parent_partner_nbr","promo_partner_nbr","promo_pricing_id","promo_market_segment","promo_expiration_dt","provenir_verification_decision_cd","provenir_credit_check_decision_cd","provenir_bank_act_check_decision_cd","provenir_application_update_decision_cd","provenir_final_terms_decision_cd","bor_customer_id","bor_phone","bor_ssn","bor_first_name","bor_middle_name","bor_last_name","bor_email","bor_address","bor_address_line2","bor_city","bor_state","bor_zip","bor_dob","bor_income","bor_us_citznshp_status","bor_residence_type","bor_mother_maiden_name","bor_mla_status","bor_provenir_ofac","app_monthly_housing_payment","app_taxable_indicator","app_privacy_consent","app_econsent","app_econsent_approval_datetime","app_econsent_approval_ip","txn_status","app_status","txn_status_datetime","decline_rsn_cd","rejection_status","expiry_datetime","approval_datetime","contract_signed_datetime","autopay_selection","autopay_method","offer_accepted_id","docusign_received_ind","ct_liabilities","provenir_liability_names","provenir_liability_amount","liability_names","liability_disb_amount","requested_amt","cash_out_amt","ct_loans_past_9m","conduent_open_loans","conduent_charge_off","conduent_dpd_xdays_past_due","loan_id","loan_onboarding_date","loan_total_funded_amount","loan_mnthly_pmt_amt","prove_ssn_match","prove_dob_match","prove_ssn_value","prove_dob_value","prove_status","prove_zipcode","prove_additionalinfo","plaid_status","file_date"]
        application_vars = ["application_id","promo_segment_id","application_datetime","merchant_id","loan_purpose","promo_code","solicitation_id","promo_parent_partner_nbr","promo_partner_nbr","promo_pricing_id","promo_market_segment","promo_expiration_dt","bor_customer_id","bor_phone","bor_ssn","bor_first_name","bor_middle_name","bor_last_name","bor_email","bor_address","bor_address_line2","bor_city","bor_state","bor_zip","bor_dob","bor_income","bor_us_citznshp_status","bor_residence_type","bor_mother_maiden_name","bor_mla_status","bor_provenir_ofac","app_monthly_housing_payment","app_taxable_indicator","app_privacy_consent","app_econsent","app_econsent_approval_datetime","app_econsent_approval_ip","approval_datetime","ct_liabilities","ct_loans_past_9m","conduent_open_loans","conduent_charge_off","conduent_dpd_xdays_past_due","prove_ssn_match","prove_dob_match","prove_ssn_value","prove_dob_value","prove_status","prove_zipcode","prove_additionalinfo","provenir_liability_names","provenir_liability_amount","app_status","app_status_datetime","decline_rsn_cd","rejection_status","expiry_datetime","contract_signed_datetime","autopay_selection","autopay_method","offer_accepted_id","liability_names","liability_disb_amount","requested_amt","cash_out_amt","loan_id","loan_onboarding_date","loan_total_funded_amount","loan_mnthly_pmt_amt","plaid_status","provenir_verification_decision_cd","provenir_credit_check_decision_cd","provenir_bank_act_check_decision_cd","provenir_application_update_decision_cd","provenir_final_terms_decision_cd","provenir_verification_response","provenir_credit_check_response","provenir_bank_account_check_response","provenir_application_update_check_response","provenir_final_terms_check_response","plaid_identity_response","file_date"]
        vendor_vars = ["transaction_id","application_id","prove_request","prove_response","plaid_identity_request","plaid_identity_response","provenir_verification_request","provenir_verification_response","provenir_credit_check_request","provenir_credit_check_response","provenir_bank_account_check_request","provenir_bank_account_check_response","provenir_final_terms_check_request","provenir_final_terms_check_response","provenir_applicaton_update_check_request","provenir_applicaton_update_check_response","file_date"]
        offer_vars = ["parent_offer_id","offer_id","application_id","transaction_id","type","term","apr","shown","autopay_ind","payment_amount","max_loan_amount","max_cash_amount","file_date"]


        txn_file_query = ("""
                            select 
                            td.transaction_id,
                            oe.segment_id as promo_segment_id,
                            td.transaction_timestamp as transaction_datetime,
                            td.merchant_id as merchant_id,
                            td.application_id as application_id,
                            ad.product_type as loan_purpose,
                            td.campaign_id as promo_code,
                            oe.solicitation_id as solicitation_id,
                            oe.agent_bank as promo_parent_partner_nbr,
                            oe.sub_agent as promo_partner_nbr,
                            oe.pricing_table as promo_pricing_id,
                            oe.market_segment as promo_market_segment,
                            oe.promo_expiry as promo_expiration_dt,
                            pr.provenir_verification_decision_cd,
                            pr.provenir_credit_check_decision_cd,
                            pr.provenir_bank_act_check_decision_cd,
                            pr.provenir_application_update_decision_cd,
                            pr.provenir_final_terms_decision_cd,
                            td.customer_id as bor_customer_id,
                            '' as bor_phone,
                            '' as bor_ssn,
                            '' as bor_first_name,
                            '' as bor_middle_name,
                            '' as bor_last_name,
                            '' as bor_email,
                            '' as bor_address,
                            '' as bor_address_line2,
                            '' as bor_city,
                            '' as bor_state,
                            '' as bor_zip,
                            '' as bor_dob,
                            '' as bor_income,
                            '' as bor_us_citznshp_status,
                            '' as bor_residence_type,
                            '' as bor_mother_maiden_name,
                            pr.bor_mla_status,
                            pr.bor_provenir_ofac,
                            case when oe.transaction_id is not null then ad.bor_housing_payment end as app_monthly_housing_payment,
                            case when oe.transaction_id is not null then not(ad.borrower_non_taxable_indicator) end as app_taxable_indicator,
                            ci.app_privacy_consent,
                            ci.app_econsent,
                            ci.app_econsent_approval_datetime,
                            ci.app_econsent_approval_ip,
                            td.status as txn_status,
                            app_status.status as app_status,
                            td.status_updated_at as txn_status_datetime,
                            td.decline_reason_code as decline_rsn_cd,
                            td.rejection_status,
                            expiry.expiry_ts as expiry_datetime,
                            approval.approval_datetime as approval_datetime,
                            case when pao.transaction_id then pao.createdAt end as contract_signed_datetime,
                            td.auto_pay_set as autopay_selection,
                            td.autopay_method as autopay_method,
                            td.selected_offer_id as offer_accepted_id,
                            case when td.agreement_vendor like '%DOCUSIGN%' then 1 end as docusign_received_ind,
                            pr.ct_liabilities,
                            pr.provenir_liability_names,
                            pr.provenir_liability_amount,
                            oval.liability_names,
                            oval.liability_disb_amount,
                            td.request_amount as requested_amt,
                            oval.cash_out_amt,
                            '' as ct_loans_past_9m,
                            '' as conduent_open_loans,
                            '' as conduent_charge_off,
                            '' as conduent_dpd_xdays_past_due,
                            case when pac.transaction_id then ld.loan_id end as loan_id,
                            case when pac.transaction_id then date(ld.effective_at) end as loan_onboarding_date,
                            case when pac.transaction_id then ld.amount end as loan_total_funded_amount,
                            od.payment_amount as loan_mnthly_pmt_amt,
                            '' as prove_ssn_match,
                            '' as prove_dob_match,
                            '' as prove_ssn_value,
                            date(prove.prove_dob) as prove_dob_value,
                            prove.prove_status as prove_status,
                            prove.prove_zipcode as prove_zipcode,
                            prove.prove_additional_info as prove_additionalinfo,
                            '' as plaid_status,
                            date(now()) as file_date

                            from data_warehouse.v72__transaction_details_v1 as td
                            left join data_warehouse.v69__application_details_v1 as ad
                                on td.application_id = ad.application_id
                            left join data_warehouse.v69__application_approval_to_funding as af
                                on td.application_id = af.application_id
                            left join data_warehouse.v69__loan_details as ld
                                on td.application_id = ld.application_id
                            left join data_warehouse.v69__prove as prove
                                on td.transaction_id = prove.transaction_id
                            left join (
                                select o.transaction_id,
                                max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(solicitation_id) end) as solicitation_id,
                                max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(segment_id) end) as segment_id,
                                max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(agent_bank) end) as agent_bank,
                                max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(sub_agent) end) as sub_agent,
                                max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(pricing_table) end) as pricing_table,
                                max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(market_segment) end) as market_segment,
                                max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(expiry) end) as promo_expiry 
                                from data_warehouse.v72__offer_evaluation as o
                                cross join
                                json_table(
                                    evaluation_data, '$[*]' Columns( 
                                        chk_type json path '$.type',
                                        solicitation_id json path '$.decision_data.metadata.campaign_details.metadata.solicitation_id',
                                        segment_id json path '$.decision_data.metadata.campaign_details.metadata.segment_id',
                                        agent_bank json path '$.decision_data.metadata.campaign_details.metadata.agent_bank',
                                        sub_agent json path '$.decision_data.metadata.campaign_details.metadata.sub_agent',
                                        pricing_table json path '$.decision_data.metadata.campaign_details.metadata.pricing_table',
                                        market_segment json path '$.decision_data.metadata.campaign_details.metadata.market_segment',
                                        expiry json path '$.decision_data.metadata.campaign_details.expiry'
                                    )
                                ) as eval
                                where o.evaluation_type = 'INITIAL_EVALUATION'
                                group by 1
                            ) as oe
                                on td.transaction_id = oe.transaction_id

                            left join (
                                select o.transaction_id,
                                max(case when evaluation_type = "INITIAL_EVALUATION" then json_unquote(expiry) end) as expiry_ts
                                from data_warehouse.v72__offer_evaluation as o
                                cross join
                                json_table(
                                    decision_data, '$' Columns( 
                                        expiry json path '$.expiry'
                                    )
                                ) as eval
                                where o.evaluation_type = 'INITIAL_EVALUATION'
                                group by 1
                            ) as expiry
                                on td.transaction_id = expiry.transaction_id
                                
                            left join (
                                select ol.transaction_id,
                                replace(decision_data ->> "$.purchase_data.amount_breakup.liabilities[*].name",'"','') as liability_names,
                                replace(decision_data ->> "$.purchase_data.amount_breakup.liabilities[*].amount",'null',0) as liability_disb_amount,
                                replace(decision_data ->> "$.purchase_data.amount_breakup.cashout.amount",'null',0) as cash_out_amt
                                from data_warehouse.v72__offer_evaluation as ol
                                where ol.evaluation_type = 'VALIDATE_EVALUATION'
                            ) as oval
                                on td.transaction_id = oval.transaction_id
                            left join (
                                select o.transaction_id,
                                max(case when type = "PROVENIR_VERIFICATION" then json_unquote(decisionCode) end) as provenir_verification_decision_cd,
                                max(case when type = "PROVENIR_CREDIT_CHECK" then json_unquote(decisionCode) end) as provenir_credit_check_decision_cd,
                                max(case when type = "PROVENIR_FINAL_TERMS" then json_unquote(decisionCode) end) as provenir_final_terms_decision_cd,
                                max(case when type = "PROVENIR_APPLICATION_UPDATE" then json_unquote(decisionCode) end) as provenir_application_update_decision_cd,
                                max(case when type = "PROVENIR_BANK_ACCOUNT_CHECK" then json_unquote(decisionCode) end) as provenir_bank_act_check_decision_cd,
                                max(case when type = "PROVENIR_MODULE_CREDIT_CHECK" then json_unquote(mla_eligible_indicator) end) as bor_mla_status,
                                max(case when type = "PROVENIR_MODULE_CREDIT_CHECK" then json_unquote(ofac_indicator) end) as bor_provenir_ofac,
                                max(case when type = "PROVENIR_CREDIT_CHECK" then length(liabilities)-length(replace(liabilities,'",',''))+1 end) as ct_liabilities,
                                max(case when type = "PROVENIR_CREDIT_CHECK" then liabilities end) as provenir_liability_names,
                                max(case when type = "PROVENIR_CREDIT_CHECK" then liabilities_balance end) as provenir_liability_amount
                                
                                
                                from data_warehouse.v72__vendor_provenir_raw as o
                                cross join
                                json_table(
                                    response_data, '$' Columns( 
                                        decisionCode json path '$.body.decision.decisionCode',
                                        mla_eligible_indicator json path '$.body.response.decision_data.decision.mla_eligible_indicator',
                                        ofac_indicator json path '$.body.response.decision_data.decision.ofac_indicator',
                                        liabilities json path '$.body.liabilities[*].companyName',
                                        liabilities_balance json path '$.body.liabilities[*].currentBalance'
                                    )
                                ) as eval
                                group by 1
                            ) as pr
                                on td.transaction_id = pr.transaction_id
                            left join (
                                SELECT 
                                transaction_id, 
                                max(case when value = 1 then 1 end) as app_privacy_consent,
                                max(case when value = 1 then 1 end) as app_econsent,
                                max(case when value = 1 and consent_type = 'terms_and_conditions' then createdAt end) as app_econsent_approval_datetime,
                                max(case when value = 1 and consent_type = 'terms_and_conditions' then ip_address end) as app_econsent_approval_ip

                                FROM data_warehouse.v72__consent_indicators
                                group by 1
                            ) as ci
                            on td.transaction_id = ci.transaction_id
                            left join (
                                select * from (
                                    select application_id, status, row_number()over(partition by application_id order by created_at desc) as rn from data_warehouse.v69__application_status) as a
                                where rn=1
                            )
                            as app_status on td.application_id = app_status.application_id
                            left join (
                                select rs.transaction_id, rs.order_id, min(oe.evaluated_at) as approval_datetime from data_warehouse.v72__v0_request_stats as rs
                                left join data_warehouse.v72__offer_evaluation as oe
                                on rs.transaction_id = oe.transaction_id or rs.order_id = oe.order_id
                                where rs.action = 'APPLICATION_SUBMIT' and rs.status_code = '200'
                                and oe.evaluation_type = 'INITIAL_EVALUATION' and oe.decision = 'APPROVED'
                                group by 1,2
                            )
                            as approval on approval.transaction_id = td.transaction_id or approval.order_id = td.order_id

                            left join (select * from data_warehouse.v69__pos_card_account_to_action where type = 'OPEN') as pao on pao.transaction_id = td.transaction_id
                            left join (select * from data_warehouse.v69__pos_card_account_to_action where type = 'AUTH_CLEAR') as pac on pac.transaction_id = td.transaction_id
                            left join data_warehouse.v72__offer_details as od on od.transaction_id = td.transaction_id and td.selected_offer_id = od.offer_id and pac.transaction_id is not null
                            where td.status is not null and td.status<>'CREATED'
                            and td.updatedAt>='"""+start_date+"""' and td.updatedAt<='"""+end_date+"""';

                            """)
        
        query_offer_file = ("""
                            select 
                            o.parent_offer_id,
                            o.offer_id,
                            o.application_id,
                            o.transaction_id,
                            o.type,
                            o.term,
                            o.apr,
                            o.shown,
                            o.is_autopay_required as autopay_ind,
                            o.payment_amount,
                            o.max_amount as max_loan_amount, 
                            "" as max_cash_amount,
                            date(now()) as file_date
                            from data_warehouse.v72__transaction_details_v1 as td
                            left join data_warehouse.v72__offer_details as o
                            on td.transaction_id = o.transaction_id
                            where td.updatedAt>='"""+start_date+"""' and td.updatedAt<='"""+end_date+"""';
                            """)

        query_application_file = ("""
                                    select 
                                    ad.application_id,
                                    oe.segment_id as promo_segment_id,
                                    ad.application_timestamp as application_datetime,
                                    txn_fst.merchant_id as merchant_id,
                                    ad.product_type as loan_purpose,
                                    txn_fst.campaign_id as promo_code,
                                    oe.solicitation_id as solicitation_id,
                                    oe.agent_bank as promo_parent_partner_nbr,
                                    oe.sub_agent as promo_partner_nbr,
                                    oe.pricing_table as promo_pricing_id,
                                    oe.market_segment as promo_market_segment,
                                    oe.expiry as promo_expiration_dt,
                                    ad.bor_customer_id,
                                    '' as bor_phone,
                                    '' as bor_ssn,
                                    '' as bor_first_name,
                                    '' as bor_middle_name,
                                    '' as bor_last_name,
                                    '' as bor_email,
                                    '' as bor_address,
                                    '' as bor_address_line2,
                                    '' as bor_city,
                                    '' as bor_state,
                                    '' as bor_zip,
                                    '' as bor_dob,
                                    '' as bor_income,
                                    '' as bor_us_citznshp_status,
                                    '' as bor_residence_type,
                                    '' as bor_mother_maiden_name,
                                    pr.bor_mla_status,
                                    pr.bor_provenir_ofac,
                                    ad.bor_housing_payment as app_monthly_housing_payment,
                                    not(ad.borrower_non_taxable_indicator) as app_taxable_indicator,
                                    ci.app_privacy_consent,
                                    ci.app_econsent,
                                    ci.app_econsent_approval_datetime,
                                    ci.app_econsent_approval_ip,
                                    approval.approval_datetime as approval_datetime,
                                    pr.ct_liabilities,
                                    '' as ct_loans_past_9m,
                                    '' as conduent_open_loans,
                                    '' as conduent_charge_off,
                                    '' as conduent_dpd_xdays_past_due,
                                    '' as prove_ssn_match,
                                    '' as prove_dob_match,
                                    '' as prove_ssn_value,
                                    date(prove.prove_dob) as prove_dob_value,
                                    prove.prove_status as prove_status,
                                    prove.prove_zipcode as prove_zipcode,
                                    prove.prove_additional_info as prove_additionalinfo,
                                    pr.provenir_liability_names,
                                    pr.provenir_liability_amount,
                                    app_status.status as app_status,
                                    app_status.created_at as app_status_datetime,
                                    txn.decline_reason_code as decline_rsn_cd,
                                    txn.rejection_status,
                                    expiry.expiry_ts as expiry_datetime,
                                    af.contract_timestamp as contract_signed_datetime,
                                    txn.auto_pay_set as autopay_selection,
                                    af.autopay_method as autopay_method,
                                    af.selected_offer_id as offer_accepted_id,
                                    oval.liability_names,
                                    oval.liability_disb_amount,
                                    txn.request_amount as requested_amt,
                                    oval.cash_out_amt,
                                    ld.loan_id as loan_id,
                                    date(ld.effective_at) as loan_onboarding_date,
                                    ld.amount as loan_total_funded_amount,
                                    od.payment_amount as loan_mnthly_pmt_amt,
                                    '' as plaid_status,
                                    pr.provenir_verification_decision_cd,
                                    pr.provenir_credit_check_decision_cd,
                                    pr.provenir_bank_act_check_decision_cd,
                                    pr.provenir_application_update_decision_cd,
                                    pr.provenir_final_terms_decision_cd,
                                    pr.provenir_verification_response,
                                    pr.provenir_credit_check_response,
                                    pr.provenir_bank_account_check_response,
                                    pr.provenir_application_update_check_response,
                                    pr.provenir_final_terms_check_response,
                                    "" as plaid_identity_response,
                                    date(now()) as file_date,
                                    txn_fst.transaction_id as first_transaction_id,
                                    txn.transaction_id as last_transaction_id                        

                                    from data_warehouse.v69__application_details_v1 as ad
                                    left join data_warehouse.v69__application_approval_to_funding as af
                                        on ad.application_id = af.application_id
                                    left join data_warehouse.v69__application_to_lender as al
                                        on ad.application_id = al.application_id
                                    left join data_warehouse.v69__loan_details as ld
                                        on ad.application_id = ld.application_id
                                    left join data_warehouse.v72__offer_details as od 
                                        on af.application_id = od.application_id and af.selected_offer_id = od.offer_id
                                    left join (
                                        select * from (
                                            select *, row_number()over(partition by application_id order by transaction_timestamp) as rn from data_warehouse.v72__transaction_details_v1) as a
                                        where rn=1 and status is not null
                                    )
                                    as txn_fst on ad.application_id = txn_fst.application_id
                                    left join data_warehouse.v69__prove as prove
                                        on txn_fst.transaction_id = prove.transaction_id
                                    left join (
                                        select o.application_id,
                                        max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(solicitation_id) end) as solicitation_id,
                                        max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(segment_id) end) as segment_id,
                                        max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(agent_bank) end) as agent_bank,
                                        max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(sub_agent) end) as sub_agent,
                                        max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(pricing_table) end) as pricing_table,
                                        max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(market_segment) end) as market_segment,
                                        max(case when json_unquote(chk_type) = "APPLICATION_BASED_CHECK" then json_unquote(expiry) end) as expiry 
                                        from data_warehouse.v72__offer_evaluation as o
                                        cross join
                                        json_table(
                                            evaluation_data, '$[*]' Columns( 
                                                chk_type json path '$.type',
                                                solicitation_id json path '$.decision_data.metadata.campaign_details.metadata.solicitation_id',
                                                segment_id json path '$.decision_data.metadata.campaign_details.metadata.segment_id',
                                                agent_bank json path '$.decision_data.metadata.campaign_details.metadata.agent_bank',
                                                sub_agent json path '$.decision_data.metadata.campaign_details.metadata.sub_agent',
                                                pricing_table json path '$.decision_data.metadata.campaign_details.metadata.pricing_table',
                                                market_segment json path '$.decision_data.metadata.campaign_details.metadata.market_segment',
                                                expiry json path '$.decision_data.metadata.campaign_details.expiry'
                                            )
                                        ) as eval
                                        where o.evaluation_type = 'INITIAL_EVALUATION'
                                        group by 1
                                    ) as oe
                                        on ad.application_id = oe.application_id
                                        

                                    left join (
                                        select o.transaction_id,
                                        max(case when evaluation_type = "INITIAL_EVALUATION" then json_unquote(expiry) end) as expiry_ts
                                        from data_warehouse.v72__offer_evaluation as o
                                        cross join
                                        json_table(
                                            decision_data, '$' Columns( 
                                                expiry json path '$.expiry'
                                            )
                                        ) as eval
                                        where o.evaluation_type = 'INITIAL_EVALUATION'
                                        group by 1
                                    ) as expiry
                                        on txn_fst.transaction_id = expiry.transaction_id
                                            
                                    left join (
                                        select ol.application_id,
                                        replace(decision_data ->> "$.purchase_data.amount_breakup.liabilities[*].name",'"','') as liability_names,
                                        replace(decision_data ->> "$.purchase_data.amount_breakup.liabilities[*].amount",'null',0) as liability_disb_amount,
                                        replace(decision_data ->> "$.purchase_data.amount_breakup.cashout.amount",'null',0) as cash_out_amt
                                        from data_warehouse.v72__offer_evaluation as ol
                                        where ol.evaluation_type = 'VALIDATE_EVALUATION'
                                    ) as oval
                                        on ad.application_id = oval.application_id

                                    left join (
                                        select o.application_id,
                                        max(case when type = "PROVENIR_VERIFICATION" then json_unquote(decisionCode) end) as provenir_verification_decision_cd,
                                        max(case when type = "PROVENIR_CREDIT_CHECK" then json_unquote(decisionCode) end) as provenir_credit_check_decision_cd,
                                        max(case when type = "PROVENIR_FINAL_TERMS" then json_unquote(decisionCode) end) as provenir_final_terms_decision_cd,
                                        max(case when type = "PROVENIR_APPLICATION_UPDATE" then json_unquote(decisionCode) end) as provenir_application_update_decision_cd,
                                        max(case when type = "PROVENIR_BANK_ACCOUNT_CHECK" then json_unquote(decisionCode) end) as provenir_bank_act_check_decision_cd,
                                        max(case when type = "PROVENIR_MODULE_CREDIT_CHECK" then json_unquote(mla_eligible_indicator) end) as bor_mla_status,
                                        max(case when type = "PROVENIR_MODULE_CREDIT_CHECK" then json_unquote(ofac_indicator) end) as bor_provenir_ofac,
                                        max(case when type = "PROVENIR_CREDIT_CHECK" then length(liabilities)-length(replace(liabilities,'",',''))+1 end) as ct_liabilities,
                                        max(case when type = "PROVENIR_CREDIT_CHECK" then liabilities end) as provenir_liability_names,
                                        max(case when type = "PROVENIR_CREDIT_CHECK" then liabilities_balance end) as provenir_liability_amount,
                                        max(case when type = "PROVENIR_VERIFICATION" then response_data end) as provenir_verification_response,
                                        max(case when type = "PROVENIR_CREDIT_CHECK" then response_data end) as provenir_credit_check_response,
                                        max(case when type = "PROVENIR_FINAL_TERMS" then response_data end) as provenir_final_terms_check_response,
                                        max(case when type = "PROVENIR_APPLICATION_UPDATE" then response_data end) as provenir_application_update_check_response,
                                        max(case when type = "PROVENIR_BANK_ACCOUNT_CHECK" then response_data end) as provenir_bank_account_check_response
                                        
                                        
                                        
                                        from data_warehouse.v72__vendor_provenir_raw as o
                                        cross join
                                        json_table(
                                            response_data, '$' Columns( 
                                                decisionCode json path '$.body.decision.decisionCode',
                                                mla_eligible_indicator json path '$.body.response.decision_data.decision.mla_eligible_indicator',
                                                ofac_indicator json path '$.body.response.decision_data.decision.ofac_indicator',
                                                liabilities json path '$.body.liabilities[*].companyName',
                                                liabilities_balance json path '$.body.liabilities[*].currentBalance'
                                            )
                                        ) as eval
                                        group by 1
                                    ) as pr
                                        on ad.application_id = pr.application_id
                                    left join (
                                        SELECT 
                                        transaction_id, 
                                        max(case when value = 1 then 1 end) as app_privacy_consent,
                                        max(case when value = 1 then 1 end) as app_econsent,
                                        max(case when value = 1 and consent_type = 'terms_and_conditions' then createdAt end) as app_econsent_approval_datetime,
                                        max(case when value = 1 and consent_type = 'terms_and_conditions' then ip_address end) as app_econsent_approval_ip

                                        FROM data_warehouse.v72__consent_indicators
                                        group by 1
                                    ) as ci
                                    on txn_fst.transaction_id = ci.transaction_id
                                    left join (
                                        select * from (
                                            select application_id, status, created_at, row_number()over(partition by application_id order by created_at desc) as rn from data_warehouse.v69__application_status) as a
                                        where rn=1
                                    )
                                    as app_status on ad.application_id = app_status.application_id
                                    left join (
                                        select * from (
                                            select *, row_number()over(partition by application_id order by status_updated_at desc) as rn from data_warehouse.v72__transaction_details_v1) as a
                                        where rn=1 and status is not null
                                    )
                                    as txn on ad.application_id = txn.application_id
                                    left join (
                                        select rs.transaction_id, rs.order_id, min(oe.evaluated_at) as approval_datetime from data_warehouse.v72__v0_request_stats as rs
                                        left join data_warehouse.v72__offer_evaluation as oe
                                        on rs.transaction_id = oe.transaction_id or rs.order_id = oe.order_id
                                        where rs.action = 'APPLICATION_SUBMIT' and rs.status_code = '200'
                                        and oe.evaluation_type = 'INITIAL_EVALUATION' and oe.decision = 'APPROVED'
                                        group by 1,2
                                    )
                                    as approval on approval.transaction_id = txn_fst.transaction_id or approval.order_id = txn_fst.order_id

                                    where txn_fst.transaction_id is not null
                                    and txn.updatedAt>='"""+start_date+"""' and txn.updatedAt<='"""+end_date+"""'
                                    ;
                                    """)
        query_vendor_file = ("""
                            select 
                            td.transaction_id,
                            td.application_id,
                            "" as prove_request,
                            "" as prove_response,
                            "" as plaid_identity_request,
                            "" as plaid_identity_response,
                            "" as provenir_verification_request,
                            "" as provenir_verification_response,
                            "" as provenir_credit_check_request,
                            "" as provenir_credit_check_response,
                            "" as provenir_bank_account_check_request,
                            "" as provenir_bank_account_check_response,
                            "" as provenir_final_terms_check_request,
                            "" as provenir_final_terms_check_response,
                            "" as provenir_applicaton_update_check_request,
                            "" as provenir_applicaton_update_check_response,
                            date(now()) as file_date
                            from data_warehouse.v72__transaction_details_v1 as td
                            where td.status is not null
                            and td.updatedAt>='"""+start_date+"""' and td.updatedAt<='"""+end_date+"""';
                            """)

        connection = ms.connect(user=DB_MYSQL_USER, password=DB_MYSQL_PASS, host=DB_MYSQL_HOST, port=DB_MYSQL_PORT, connection_timeout =1000)
        txn_file_data = pd.read_sql(txn_file_query, connection)
        application_file_data = pd.read_sql(query_application_file, connection)
        vendor_file_data = pd.read_sql(query_vendor_file, connection)
        offer_file_data = pd.read_sql(query_offer_file, connection)
        connection.close()
        Number_Of_Records = txn_file_data.shape[0]
        write_log_count_file_ucl(start_date, end_date,Number_Of_Records,0, 'Transaction')

        ## Add PII data
        #Transaction file
        for i in range(0,len(txn_file_data)):
            app_id = txn_file_data.loc[i,"application_id"]
            txn_id = txn_file_data.loc[i,"transaction_id"]
            
            if (pd.notna(app_id)) :
                txn_file_data.loc[i,["bor_phone","bor_ssn","bor_first_name","bor_middle_name","bor_last_name","bor_email","bor_address","bor_address_line2","bor_city","bor_state","bor_zip","bor_dob","bor_income","bor_us_citznshp_status","bor_residence_type","bor_mother_maiden_name"]] = get_pii_data(url,app_id)
            
            prove_ssn, prove_dob = get_prove_ssn_dob(txn_id,"transaction_id")
            txn_file_data.loc[i,["prove_ssn_match","prove_dob_match"]] = [get_match_ind(txn_file_data.loc[i,"bor_ssn"],prove_ssn),get_match_ind(txn_file_data.loc[i,"bor_dob"],prove_dob)]

        #Application file
        for i in range(0,len(application_file_data)):
            app_id = application_file_data.loc[i,"application_id"]
            txn_id = application_file_data.loc[i,"first_transaction_id"]
            if (pd.notna(app_id)):
                application_file_data.loc[i,["bor_phone","bor_ssn","bor_first_name","bor_middle_name","bor_last_name","bor_email","bor_address","bor_address_line2","bor_city","bor_state","bor_zip","bor_dob","bor_income","bor_us_citznshp_status","bor_residence_type","bor_mother_maiden_name"]] = get_pii_data(url,app_id)

            prove_ssn, prove_dob = get_prove_ssn_dob(txn_id,"transaction_id")
            txn_file_data.loc[i,["prove_ssn_match","prove_dob_match"]] = [get_match_ind(txn_file_data.loc[i,"bor_ssn"],prove_ssn),get_match_ind(txn_file_data.loc[i,"bor_dob"],prove_dob)]

        ## Get vendor responses fron API
        for i in range(0,len(vendor_file_data)):
            id = vendor_file_data.loc[i,"transaction_id"]
            id_type = 'transaction_id'

            params = {"id": id, "id_type": id_type}
            api_url = url+"/application/logs/vendor/"
            resp = get_api_data(api_url, params=params)
            
            prove_request, prove_response = get_prove_data(resp)
            plaid_identity_request, plaid_identity_response = get_plaid_data(resp)

            provenir_verification_request, provenir_verification_response, \
            provenir_credit_check_request, provenir_credit_check_response, \
            provenir_bank_account_check_request, provenir_bank_account_check_response, \
            provenir_final_terms_check_request, provenir_final_terms_check_response, \
            provenir_applicaton_update_check_request, provenir_applicaton_update_check_response = get_provenir_data(resp)

            vendor_file_data.loc[i,["prove_request", "prove_response", "plaid_identity_request", "plaid_identity_response", "provenir_verification_request", "provenir_verification_response", "provenir_credit_check_request", "provenir_credit_check_response", "provenir_bank_account_check_request", "provenir_bank_account_check_response", "provenir_final_terms_check_request", "provenir_final_terms_check_response", "provenir_applicaton_update_check_request", "provenir_applicaton_update_check_response"]] = [prove_request, prove_response, plaid_identity_request, plaid_identity_response, provenir_verification_request, provenir_verification_response, provenir_credit_check_request, provenir_credit_check_response, provenir_bank_account_check_request, provenir_bank_account_check_response, provenir_final_terms_check_request, provenir_final_terms_check_response, provenir_applicaton_update_check_request, provenir_applicaton_update_check_response]
        
        ## Format variables
        # Change None to NaN
        # # none_to_nan(txn_file_data)
        # # none_to_nan(application_file_data)
        # none_to_nan(vendor_file_data)
        none_to_nan(offer_file_data)

        # Format to integer
        txn_file_data.app_taxable_indicator = txn_file_data.app_taxable_indicator.astype('Int64')

        ## Final subset
        txn_file_final = txn_file_data[txn_vars]
        application_file_final = application_file_data[application_vars]
        vendor_file_final = vendor_file_data[vendor_vars]
        offer_file_final = offer_file_data[offer_vars]

        ## Write files

        data_dfs2 = [txn_file_final,application_file_final,vendor_file_final,offer_file_final]

        filenames = ['ucl_transaction.txt','ucl_application.txt','ucl_vendor_response.txt','ucl_offer_details.txt']
        abc_filenames = ['ucl_transaction.abc','ucl_application.abc','ucl_vendor_response.abc','ucl_offer_details.abc']

        write_files(data_dfs2, filenames, abc_filenames, folder_name)

        ## UPDATE SUCCESS FLAG as 1
        file_path = 'Logs_UCL.csv'
        df = pd.read_csv(file_path)

        # Specify the column you want to update
        column_to_update = 'Success_Flag'

        # Get the index of the last row
        last_row_index = df.index[-1]

        # Update the value of the last record in the specified column
        new_value = 1  # Replace 'New Value' with your desired value
        df.at[last_row_index, column_to_update] = new_value

        df.to_csv(file_path, index=False)

        # sentry.send_message("Merchant reports generated successfully ✅")

        return "UCL reports generated successfully"

    except Exception as generic_error:
        # debugger
        # traceback.print_exc()
        error_case(generic_error, "Main Call")
        # raise generic_error

In [51]:
os.chdir('/Users/anahita/Library/CloudStorage/OneDrive-StreamSourceTechnologies/Documents/Projects/FNBO UCL/Data/Data file transfer')
main_func("Manual","test","2025-03-15","2025-04-06")

📂 current_directory ➟  .
Folder 'ucl_data_files' already exists.
📂 new_folder_path ➟  ./ucl_data_files
📂 folder_name ➟  2025_03_15_2025_04_06
Folder '2025_03_15_2025_04_06' already exists.


In [8]:
id_type = 'transaction_id'

params = {"id": "2utFwDrRlcJYoJOaEnxJoCKvECH", "id_type": id_type}
api_url = url+"/application/logs/vendor/"
resp = get_api_data(api_url, params=params)

In [9]:
prove_request, prove_response = get_prove_data(resp)

In [24]:
prove_response

'{"body": {"requestId": "test_2utFwDrRlcJYoJOaEnxJoCKvECH_938d70ac-eef4-433e-a10e-3b409b00827b", "status": 0, "description": "Success.", "response": {"transactionId": "12223845687", "phoneNumber": "12001022006", "lineType": "Mobile", "carrier": "Sprint", "countryCode": "US", "reasonCodes": ["OU"], "individual": {"firstName": "Frasquito", "lastName": "Mylan", "addresses": [{"address": "1 Dwight Court", "extendedAddress": "", "city": "Ridgely", "region": "MD", "postalCode": "21684"}], "ssn": "666399741", "dob": "1987-03-22"}}}, "status_code": 200, "duration": 422, "headers": {"date": "Thu, 27 Mar 2025 07:23:53 GMT", "content-type": "application/json", "content-length": "482", "connection": "close", "cache-control": "no-cache, no-store, max-age=0, must-revalidate", "pragma": "no-cache", "expires": "0", "x-content-type-options": "nosniff", "x-frame-options": "DENY", "x-xss-protection": "0", "referrer-policy": "no-referrer"}}'

In [39]:
prove_request_2, prove_response_2 = get_prove_data(get_api_data(api_url, params={"id": "2utFwDrRlcJYoJOaEnxJoCKvECH", "id_type": id_type}))
if pd.notna(prove_response_2):
    prove_response_js = oj.loads(prove_response_2)
    prove_ssn = prove_response_js.get("body",{}).get("response",{}).get("individual",{}).get("ssn",{})
    prove_dob = prove_response_js.get("body",{}).get("response",{}).get("individual",{}).get("dob",{})
else:
    prove_ssn = ""
    prove_dob = ""

In [40]:
prove_ssn,prove_dob

('666399741', '1987-03-22')

In [46]:
get_match_ind("43543545435",prove_ssn),get_match_ind("1987-03-21",prove_dob)

(0, 0)

In [14]:
get_api_data(api_url, params={"id": "2uZj0dQfiPBTVV7wi7LqRxyrtN9", "id_type": id_type})

{'status': 'Success',
 'status_code': 200,
 'message': 'External Logs fetched successfully!!.',
 'response': {}}